In [58]:
import pandas as pd
import numpy as np
import nltk
import string
import regex as re

import umap
import hdbscan
import bertopic
import sentence_transformers
import sklearn
import numpy as np

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from umap import UMAP
from hdbscan import HDBSCAN


nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

True

In [108]:
df = pd.read_parquet("dlr_prototype/metadata.parquet")
plain_text = df["plain_text"].to_list()
display(df)

plain_text = [re.sub(r"\n", " ", sentence) for sentence in plain_text]

plain_text = ["".join(ch for ch in doc if (ch.isalpha() or ch == " ")) for doc in plain_text]
plain_text = [re.sub(r"^\s*\.*","", doc) for doc in plain_text]
plain_text = [re.sub(r" +"," ", doc) for doc in plain_text]

eng_stopword_list = nltk.corpus.stopwords.words('english')
# ger_stopword_list = nltk.corpus.stopwords.words('german')
# special_words = ["TU", "Graz", "University", "Universität", "Überblick", "die", "sie", "der", "das", "er"]
#stopwordlist = eng_stopword_list + ger_stopword_list + special_words

stopwordlist = eng_stopword_list
stemmer = SnowballStemmer(language='english')

plain_text = [" ".join(word for word in doc.split(" ") if (word not in stopwordlist)) for doc in plain_text]

plain_text = [" ".join(stemmer.stem(word.lower()) for word in doc.split(" ")) for doc in plain_text]

print(len(plain_text))

,id,record_id,warc_date,warc_ip,url,url_scheme,url_path,url_params,url_query,url_fragment,...,title,plain_text,language,microdata,json-ld,outgoing_links,locations,domain_labels,domain_labels_en,domain_label
0,NaN,c5587213-5d4f-43a2-b357-cc4b30811c0f,2024-02-06T08:02:48Z,2.18.36.223,https://www.aljazeera.com/features/2023/11/15/...,https,/features/2023/11/15/two-months-after-flood-in...,,,,...,"Two months after flood in Libya’s Derna, menta...",00006000 Two months after flood in Libya’s Der...,eng,None,"[{'@context': 'https://schema.org', '@type': '...",[https://www.aljazeera.com/tag/coronavirus-pan...,"[{'Libya': [[28.0, 17.0, 'LY']]}, {'Derna': [[...","['/en/News/', '/en/Regional/Middle_East/Qatar/...","['/en/News/', '/en/Regional/Middle_East/Qatar/...",News
1,NaN,0761818b-c446-4a4b-be81-d86ed19f1d43,2024-02-06T08:02:48Z,2.18.36.223,https://www.aljazeera.com/news/2023/11/15/two-...,https,/news/2023/11/15/two-months-after-flood-in-lib...,,,,...,None,vX7OA_gE4X#3mlƌٵ$3@!2{7q_>}HBL6 0::gwt#;v...,zho,None,None,None,None,"['/en/News/', '/en/Regional/Middle_East/Qatar/...","['/en/News/', '/en/Regional/Middle_East/Qatar/...",News
2,NaN,10996f7f-db0c-4c04-b834-fe61cdea6ff3,2024-02-06T08:02:44Z,2.18.36.223,https://www.aljazeera.com/news/2023/11/6/kenya...,https,/news/2023/11/6/kenya-flooding-kills-15-displa...,,,,...,Death and displacement as floods hit East Afri...,00006000 Death and displacement as floods hit ...,eng,None,"[{'@context': 'https://schema.org', '@type': '...",[https://www.aljazeera.com/tag/coronavirus-pan...,"[{'Africa': [[5.40283, -74.86718, 'CO'], [1.07...","['/en/News/', '/en/Regional/Middle_East/Qatar/...","['/en/News/', '/en/Regional/Middle_East/Qatar/...",News
3,NaN,07ad77cf-988b-4de5-a37a-34f550672579,2024-02-06T08:03:02Z,34.111.72.3,https://apnews.com/article/australia-tropical-...,https,/article/australia-tropical-cyclone-kirrily-fl...,,,,...,More flooding forecast for Australia's northea...,35a More flooding forecast for Australia's nor...,eng,None,"[{'@context': 'http://schema.org', '@type': 'N...","[https://apnews.com/world-news, https://apnews...","[{'Australia': [[-19.85, -62.51667, 'BO'], [-1...",['/en/News/'],['/en/News/'],News
4,NaN,d7252d91-21b1-451f-b345-67d5b6c25c66,2024-02-06T08:02:59Z,34.111.72.3,https://apnews.com/article/flooding-northeast-...,https,/article/flooding-northeast-illinois-kankakee-...,,,,...,Evacuations underway in northeast Illinois aft...,1e07 Evacuations underway in northeast Illinoi...,eng,None,"[{'@context': 'http://schema.org', '@type': 'N...","[https://apnews.com/world-news, https://apnews...","[{'Illinois': [[-34.14437, 147.05553, 'AU'], [...",['/en/News/'],['/en/News/'],News
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,NaN,a07ebb40-d19f-4ef0-80f1-0d80663997de,2024-02-06T08:02:46Z,104.18.155.21,https://www.who.int/europe/news/item/22-05-202...,https,/europe/news/item/22-05-2023-flooding-in-north...,,,,...,\n\tFlooding in northern Italy and central Eur...,25be Flooding in northern Italy and central Eu...,eng,None,None,"[https://www.who.int/, https://www.who.int/, h...","[{'Italy': [[24.52011, 89.22808, 'BD'], [-25.1...",['/en/Society/Government/Multilateral/United_N...,"['/en/Regional/Europe/Government', '/en/Societ...",Regional
429,NaN,876c17d7-2919-418c-ab19-4db4d98a8761,2024-02-06T08:02:35Z,104.18.155.21,https://www.who.int/health-topics/floods,https,/health-topics/floods,,,,...,\n\tFloods\n,19 Floods\n • World Health Organization Globa...,eng,None,None,"[https://www.who.int/, https://www.who.int/, h...","[{'Europe': [[46.7967, 9.821670000000001, 'CH'...",['/en/Society/Government/Multilateral/United_N...,"['/en/Regional/Europe/Government', '/en/Societ...",Regional
430,NaN,16c278ff-41d0-4c8a-8adc-8a65dae2da49,2024-02-06T08:01:43Z,185.15.59.224,https://en.wikipedia.org/wiki/Earth_observation,https,/wiki/Earth_observation,,,,...,Earth observation - Wikipedia,Jump to content\nMain menu\nNavigation\n • Ma...,eng,None,"[{'@cont

433
two month flood libya derna mental anguish still take live flood al jazeera skip link skip content play live navig menu news middl east africa asia us canada latin america europ asia pacif israel war gaza featur opinion video economi ukrain war coronavirus climat crisi investig interact in pictur scienc technolog sport podcast play live featuresflood two month flood libya derna mental anguish still take live the damag left behind physic peopl mourn lost love one deal trauma hamdi cheikh surviv dead storm hit libya react speak journalist derna hamdi cheikh surviv dead storm hit libya speak journalist derna libya septemb zohra bensemrareut byislam alatrash publish on nov nov derna libya about month flood devast libya eastern citi derna dr khale alshaari discov neighbour he taken life near floodravag home his death isol event in week follow catastroph derna resid die suicid keep read list item list a real hell the climat refuge libya flood region strife list east libya command haftar 

In [100]:
umap_model = UMAP(n_neighbors=30, n_components=2, min_dist=0.01)
hdbscan_model = HDBSCAN(min_cluster_size=20, min_samples=2, prediction_data=True, cluster_selection_method="leaf")

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    language='english',
    #calculate_probabilities=True,
    verbose=True,
    hdbscan_model = hdbscan_model
)
''
topics, probs = model.fit_transform(plain_text)

2024-05-24 15:51:50,367 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/14 [00:00<?, ?it/s]

2024-05-24 15:51:55,193 - BERTopic - Embedding - Completed ✓
2024-05-24 15:51:55,193 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-24 15:51:59,871 - BERTopic - Dimensionality - Completed ✓
2024-05-24 15:51:59,872 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-05-24 15:51:59,888 - BERTopic - Cluster - Completed ✓
2024-05-24 15:51:59,894 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-24 15:52:00,203 - BERTopic - Representation - Completed ✓


In [103]:
print(model.get_topic_info(), "\n")

fig = model.visualize_documents(plain_text)
html = fig.to_html()
with open("Test.html", "w", encoding="utf-8") as f:
    f.write(html)

   Topic  Count                        Name  \
0     -1    146    -1_map_global_imag_flood   
1      0     97     0_flood_the_river_water   
2      1     79       1_fire_burn_imag_nasa   
3      2     48     2_storm_imag_earth_nasa   
4      3     36     3_applic_like_view_read   
5      4     27  4_earth_observ_our_program   

                                      Representation  \
0  [map, global, imag, flood, water, temperatur, ...   
1  [flood, the, river, water, imag, rain, peopl, ...   
2  [fire, burn, imag, nasa, smoke, earth, observa...   
3  [storm, imag, earth, nasa, cyclon, earthquak, ...   
4  [applic, like, view, read, open, stori, focus,...   
5  [earth, observ, our, program, we, work, scienc...   

                                 Representative_Docs  
0  [bb sea surfac temperatur atmospher heat human...  
1  [educ sign in donat articl articl flood flood ...  
2  [bb heat fire scorch northern canada atmospher...  
3  [bb hurrican lee travers warm atlant atmospher...  
4 

In [130]:
topics_keywords = model.get_topics()
for topic_id, keywords in topics_keywords.items():
    print(f"Topic {topic_id}: {keywords[:10]}")

Topic -1: [('map', 0.07292657290025173), ('global', 0.06840914749275022), ('imag', 0.06818899070712639), ('flood', 0.036956778875619024), ('water', 0.035551403717375765), ('temperatur', 0.030160828729197005), ('surfac', 0.028373205848661293), ('the', 0.027022248969147674), ('data', 0.023385411960636305), ('land', 0.022780826837585727)]
Topic 0: [('flood', 0.08324531227590931), ('the', 0.03504593455078729), ('river', 0.03265348237815418), ('water', 0.028930325241586344), ('imag', 0.027968637658069624), ('rain', 0.025579402040615432), ('peopl', 0.02248370163135259), ('earth', 0.022380067533974656), ('storm', 0.021477262698741947), ('nasa', 0.02063578496864904)]
Topic 1: [('fire', 0.13161676797101657), ('burn', 0.05435435120060174), ('imag', 0.05106201067354374), ('nasa', 0.04499390281683188), ('smoke', 0.0442112371103561), ('earth', 0.04407570119679861), ('observatori', 0.04004332349473069), ('wildfir', 0.03891304548428424), ('access', 0.03192887098589918), ('day', 0.030941417982229113)]

In [121]:
new_list = []
for elem in topics:
    if elem == -1:
        new_list.append("none")
    if elem == 0:
        new_list.append("flood")
    if elem == 1:
        new_list.append("fire")
    if elem == 2:
        new_list.append("storm")
    if elem == 3:
        new_list.append("applic")
    if elem == 4:
        new_list.append("earth")

In [122]:
df = pd.read_parquet("dlr_prototype/metadata.parquet")

In [123]:
df['topic'] = new_list

In [129]:
display(df[df["topic"] != "none"])

,id,record_id,warc_date,warc_ip,url,url_scheme,url_path,url_params,url_query,url_fragment,...,plain_text,language,microdata,json-ld,outgoing_links,locations,domain_labels,domain_labels_en,domain_label,topic
0,NaN,c5587213-5d4f-43a2-b357-cc4b30811c0f,2024-02-06T08:02:48Z,2.18.36.223,https://www.aljazeera.com/features/2023/11/15/...,https,/features/2023/11/15/two-months-after-flood-in...,,,,...,00006000 Two months after flood in Libya’s Der...,eng,None,"[{'@context': 'https://schema.org', '@type': '...",[https://www.aljazeera.com/tag/coronavirus-pan...,"[{'Libya': [[28.0, 17.0, 'LY']]}, {'Derna': [[...","['/en/News/', '/en/Regional/Middle_East/Qatar/...","['/en/News/', '/en/Regional/Middle_East/Qatar/...",News,flood
2,NaN,10996f7f-db0c-4c04-b834-fe61cdea6ff3,2024-02-06T08:02:44Z,2.18.36.223,https://www.aljazeera.com/news/2023/11/6/kenya...,https,/news/2023/11/6/kenya-flooding-kills-15-displa...,,,,...,00006000 Death and displacement as floods hit ...,eng,None,"[{'@context': 'https://schema.org', '@type': '...",[https://www.aljazeera.com/tag/coronavirus-pan...,"[{'Africa': [[5.40283, -74.86718, 'CO'], [1.07...","['/en/News/', '/en/Regional/Middle_East/Qatar/...","['/en/News/', '/en/Regional/Middle_East/Qatar/...",News,flood
9,NaN,c6ba1d93-e20e-4df2-b414-6654bc9afe8b,2024-02-06T08:02:54Z,199.232.16.81,https://www.bbc.com/news/world-asia-66197937,https,/news/world-asia-66197937,,,,...,BBC Homepage\n • Skip to content\n • Accessi...,fra,None,"[{'@context': 'http://schema.org', '@type': 'R...","[https://www.bbc.com, https://www.bbc.co.uk/ac...","[{'Ukraine': [[49.0, 32.0, 'UA'], [50.76744, 2...",None,None,None,flood
10,NaN,95c10562-b3bd-4570-a163-4375c689b6ca,2024-02-06T08:02:46Z,104.18.4.110,https://www.britannica.com/event/Libya-floodin...,https,/event/Libya-flooding-of-2023,,,,...,"7ff9 Libya flooding of 2023 | Storm Daniel, Fl...",eng,None,[{'headline': 'Libya flooding of 2023 | Storm ...,[https://subscription.britannica.com/subscribe...,"[{'Libya': [[28.0, 17.0, 'LY']]}, {'Derna': [[...",['/en/Kids_and_Teens/School_Time/Reference_Too...,['/en/Kids_and_Teens/School_Time/Reference_Too...,Kids_and_Teens,flood
12,NaN,c98e4e8b-a5ce-4b63-b749-881d8c09c48d,2024-02-06T08:04:42Z,136.156.153.127,https://climate.copernicus.eu/esotc/2021/flood...,https,/esotc/2021/flooding-july,,,,...,a03d4 Flooding in Europe | Copernicus skip to ...,eng,None,None,"[https://www.ecmwf.int/, https://www.copernicu...","[{'Europe': [[46.7967, 9.821670000000001, 'CH'...",None,None,None,flood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,NaN,a3ebec93-890b-4e25-8c5c-dcc66d56f0b5,2024-02-06T08:02:42Z,104.96.130.103,https://www.unicef.org/press-releases/more-15-...,https,/press-releases/more-15-million-children-risk-...,,,,...,0000C000 More than 1.5 million children at ris...,eng,None,None,[https://www.unicef.org/emergencies/children-g...,"[{'Gaza': [[-37.77792, 142.45836, 'AU'], [-24....","['/en/Society/Issues/Economic/Foreign_Aid/UN',...","['/en/Society/Issues/Economic/Foreign_Aid/UN',...",Society,flood
427,NaN,9659d9ae-9871-4011-84e5-2b5adb8510fe,2024-02-06T08:02:47Z,104.18.155.21,https://www.emro.who.int/lby/floods/index.html,https,/lby/floods/index.html,,,,...,52 Server ErrorServer Error 403 0,unknown,None,None,None,None,['/en/Society/Government/Multilateral/United_N...,"['/en/Regional/Europe/Government', '/en/Societ...",Regional,fire
428,NaN,a07ebb40-d19f-4ef0-80f1-0d80663997de,2024-02-06T08:02:46Z,104.18.155.21,https://www.who.int/europe/news/item/22-05-202...,https,/europe/news/item/22-05-2023-flooding-in-north...,,,,...,25be Flooding in northern Italy and central Eu...,eng,None,None,"[https://www.who.int/, https://www.who.int/, h...","[{'Italy': [[24.52011, 89.22808, 'BD'], [-25.1...",['/en/Society/Government/Multilateral/United_N...,"['/en/Regional/Europe/Government', '/en/Societ...",Regional,flood
431,NaN,3e49356a-97d4-40f3-a569-61226277f13f,2024-02-06T08:02:35Z,185.15.59.224,https://en.wikipedia.org/wiki/Floo

In [125]:
df.to_parquet('metadata.parquet')